# Build your first Flow
* We’ll walk through creating a CrewAI Flow that generates a comprehensive learning guide on any topic.

## Step 1: Create a new Flow project
In terminal:
* `crewai create flow guide_creator_flow`
* `cd guide_creator_flow`

This command sets up a scaffolded project with all the necessary directories and template files for your flow.

## Step 2: Review the structure of the template
```
text
guide_creator_flow/
├── .gitignore
├── pyproject.toml
├── README.md
├── .env
├── main.py
├── crews/
│   └── poem_crew/
│       ├── config/
│       │   ├── agents.yaml
│       │   └── tasks.yaml
│       └── poem_crew.py
└── tools/
    └── custom_tool.py
```

This structure provides a clear separation between different components of your flow:
* **The main flow logic** in the main.py file.
* Specialized crews in the crews directory.
* Custom tools in the tools directory.

## Step 3: Create a new Crew
`crewai flow add-crew content-crew`

This command automatically creates the content-crew with the necessary directories and template files. 

This content-crew will be responsible for writing and reviewing sections of our guide, working within the overall flow orchestrated by our main application.

## Step 4: Configure the Crew
Now we will customize the files of the content-crew. 
* We’ll set up two specialized agents - a writer and a reviewer - that will collaborate to create high-quality content for our guide.

First, update the agents configuration file to define our content creation team:

```yaml
# src/guide_creator_flow/crews/content_crew/config/agents.yaml
content_writer:
  role: >
    Educational Content Writer
  goal: >
    Create engaging, informative content that thoroughly explains the assigned topic
    and provides valuable insights to the reader
  backstory: >
    You are a talented educational writer with expertise in creating clear, engaging
    content. You have a gift for explaining complex concepts in accessible language
    and organizing information in a way that helps readers build their understanding.
  llm: openai/gpt-4o-mini

content_reviewer:
  role: >
    Educational Content Reviewer and Editor
  goal: >
    Ensure content is accurate, comprehensive, well-structured, and maintains
    consistency with previously written sections
  backstory: >
    You are a meticulous editor with years of experience reviewing educational
    content. You have an eye for detail, clarity, and coherence. You excel at
    improving content while maintaining the original author's voice and ensuring
    consistent quality across multiple sections.
  llm: openai/gpt-4o-mini
```

Next, update the tasks configuration file to define the specific writing and reviewing tasks:

```yaml
# src/guide_creator_flow/crews/content_crew/config/tasks.yaml
write_section_task:
  description: >
    Write a comprehensive section on the topic: "{section_title}"

    Section description: {section_description}
    Target audience: {audience_level} level learners

    Your content should:
    1. Begin with a brief introduction to the section topic
    2. Explain all key concepts clearly with examples
    3. Include practical applications or exercises where appropriate
    4. End with a summary of key points
    5. Be approximately 500-800 words in length

    Format your content in Markdown with appropriate headings, lists, and emphasis.

    Previously written sections:
    {previous_sections}

    Make sure your content maintains consistency with previously written sections
    and builds upon concepts that have already been explained.
  expected_output: >
    A well-structured, comprehensive section in Markdown format that thoroughly
    explains the topic and is appropriate for the target audience.
  agent: content_writer

review_section_task:
  description: >
    Review and improve the following section on "{section_title}":

    {draft_content}

    Target audience: {audience_level} level learners

    Previously written sections:
    {previous_sections}

    Your review should:
    1. Fix any grammatical or spelling errors
    2. Improve clarity and readability
    3. Ensure content is comprehensive and accurate
    4. Verify consistency with previously written sections
    5. Enhance the structure and flow
    6. Add any missing key information

    Provide the improved version of the section in Markdown format.
  expected_output: >
    An improved, polished version of the section that maintains the original
    structure but enhances clarity, accuracy, and consistency.
  agent: content_reviewer
  context:
    - write_section_task
```

**Note how the context parameter in the review task creates a workflow where the reviewer has access to the writer’s output.**

Now, update the crew implementation file to define how our agents and tasks work together:

In [ ]:
# src/guide_creator_flow/crews/content_crew/content_crew.py
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List

@CrewBase
class ContentCrew():
    """Content writing crew"""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def content_writer(self) -> Agent:
        return Agent(
            config=self.agents_config['content_writer'], # type: ignore[index]
            verbose=True
        )

    @agent
    def content_reviewer(self) -> Agent:
        return Agent(
            config=self.agents_config['content_reviewer'], # type: ignore[index]
            verbose=True
        )

    @task
    def write_section_task(self) -> Task:
        return Task(
            config=self.tasks_config['write_section_task'] # type: ignore[index]
        )

    @task
    def review_section_task(self) -> Task:
        return Task(
            config=self.tasks_config['review_section_task'], # type: ignore[index]
            context=[self.write_section_task()]
        )

    @crew
    def crew(self) -> Crew:
        """Creates the content writing crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )

While this crew can function independently, in our flow it will be orchestrated as part of a larger system.

## Step 5: Create the Flow
We’ll combine regular Python code, direct LLM calls, and our content creation crew into a cohesive system.

First, let’s create our flow in the main.py file:

In [ ]:
#!/usr/bin/env python
import json
import os
from typing import List, Dict
from pydantic import BaseModel, Field
from crewai import LLM
from crewai.flow.flow import Flow, listen, start
from guide_creator_flow.crews.content_crew.content_crew import ContentCrew

# Define our models for structured data
class Section(BaseModel):
    title: str = Field(description="Title of the section")
    description: str = Field(description="Brief description of what the section should cover")

class GuideOutline(BaseModel):
    title: str = Field(description="Title of the guide")
    introduction: str = Field(description="Introduction to the topic")
    target_audience: str = Field(description="Description of the target audience")
    sections: List[Section] = Field(description="List of sections in the guide")
    conclusion: str = Field(description="Conclusion or summary of the guide")

# Define our flow state
class GuideCreatorState(BaseModel):
    topic: str = ""
    audience_level: str = ""
    guide_outline: GuideOutline = None
    sections_content: Dict[str, str] = {}

class GuideCreatorFlow(Flow[GuideCreatorState]):
    """Flow for creating a comprehensive guide on any topic"""

    @start()
    def get_user_input(self):
        """Get input from the user about the guide topic and audience"""
        print("\n=== Create Your Comprehensive Guide ===\n")

        # Get user input
        self.state.topic = input("What topic would you like to create a guide for? ")

        # Get audience level with validation
        while True:
            audience = input("Who is your target audience? (beginner/intermediate/advanced) ").lower()
            if audience in ["beginner", "intermediate", "advanced"]:
                self.state.audience_level = audience
                break
            print("Please enter 'beginner', 'intermediate', or 'advanced'")

        print(f"\nCreating a guide on {self.state.topic} for {self.state.audience_level} audience...\n")
        return self.state

    @listen(get_user_input)
    def create_guide_outline(self, state):
        """Create a structured outline for the guide using a direct LLM call"""
        print("Creating guide outline...")

        # Initialize the LLM
        llm = LLM(model="openai/gpt-4o-mini", response_format=GuideOutline)

        # Create the messages for the outline
        messages = [
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": f"""
            Create a detailed outline for a comprehensive guide on "{state.topic}" for {state.audience_level} level learners.

            The outline should include:
            1. A compelling title for the guide
            2. An introduction to the topic
            3. 4-6 main sections that cover the most important aspects of the topic
            4. A conclusion or summary

            For each section, provide a clear title and a brief description of what it should cover.
            """}
        ]

        # Make the LLM call with JSON response format
        response = llm.call(messages=messages)

        # Parse the JSON response
        outline_dict = json.loads(response)
        self.state.guide_outline = GuideOutline(**outline_dict)

        # Ensure output directory exists before saving
        os.makedirs("output", exist_ok=True)

        # Save the outline to a file
        with open("output/guide_outline.json", "w") as f:
            json.dump(outline_dict, f, indent=2)

        print(f"Guide outline created with {len(self.state.guide_outline.sections)} sections")
        return self.state.guide_outline

    @listen(create_guide_outline)
    def write_and_compile_guide(self, outline):
        """Write all sections and compile the guide"""
        print("Writing guide sections and compiling...")
        completed_sections = []

        # Process sections one by one to maintain context flow
        for section in outline.sections:
            print(f"Processing section: {section.title}")

            # Build context from previous sections
            previous_sections_text = ""
            if completed_sections:
                previous_sections_text = "# Previously Written Sections\n\n"
                for title in completed_sections:
                    previous_sections_text += f"## {title}\n\n"
                    previous_sections_text += self.state.sections_content.get(title, "") + "\n\n"
            else:
                previous_sections_text = "No previous sections written yet."

            # Run the content crew for this section
            result = ContentCrew().crew().kickoff(inputs={
                "section_title": section.title,
                "section_description": section.description,
                "audience_level": self.state.audience_level,
                "previous_sections": previous_sections_text,
                "draft_content": ""
            })

            # Store the content
            self.state.sections_content[section.title] = result.raw
            completed_sections.append(section.title)
            print(f"Section completed: {section.title}")

        # Compile the final guide
        guide_content = f"# {outline.title}\n\n"
        guide_content += f"## Introduction\n\n{outline.introduction}\n\n"

        # Add each section in order
        for section in outline.sections:
            section_content = self.state.sections_content.get(section.title, "")
            guide_content += f"\n\n{section_content}\n\n"

        # Add conclusion
        guide_content += f"## Conclusion\n\n{outline.conclusion}\n\n"

        # Save the guide
        with open("output/complete_guide.md", "w") as f:
            f.write(guide_content)

        print("\nComplete guide compiled and saved to output/complete_guide.md")
        return "Guide creation completed successfully"

def kickoff():
    """Run the guide creator flow"""
    GuideCreatorFlow().kickoff()
    print("\n=== Flow Complete ===")
    print("Your comprehensive guide is ready in the output directory.")
    print("Open output/complete_guide.md to view it.")

def plot():
    """Generate a visualization of the flow"""
    flow = GuideCreatorFlow()
    flow.plot("guide_creator_flow")
    print("Flow visualization saved to guide_creator_flow.html")

if __name__ == "__main__":
    kickoff()

## Let's explain the following code in simple terms first, and then line by line

#### Big Picture  
This code builds a system (called a *flow*) that helps create a full, detailed **guide** on any topic. It gets the topic from the user, plans the guide structure with AI, writes the content for each section using a team of AI workers (*ContentCrew*), and saves everything nicely into files.

#### Step-by-Step in plain English

1. **Set up the data formats**:  
   - We define two data models (`Section` and `GuideOutline`) using **Pydantic**.  
     Think of these like blueprints telling the system *"this is what a guide or a section should look like."*

2. **Create a memory (state) class**:  
   - We create a `GuideCreatorState` that holds the *topic*, *audience level*, *the guide outline*, and *the actual written sections*.  
     This way, information can be passed between steps.

3. **Define the Flow**:  
   A *Flow* is like a recipe, with different steps that happen in order.

   - **Step 1** (`@start` decorator):  
     - **Ask the user** what guide they want to create and who the guide is for (beginner, intermediate, advanced).
   
   - **Step 2** (`@listen(get_user_input)` decorator):  
     - **Use an AI (LLM)** to create a guide outline automatically based on the user's input.
     - The AI answers with a **structured JSON** that fits our `GuideOutline` model.
     - This outline is **saved into a file**.

   - **Step 3** (`@listen(create_guide_outline)` decorator):  
     - For **each section** of the outline:
       - Use a **content creation AI crew** (ContentCrew) to write the section.
       - Keep track of the content as it's being written.
     - After all sections are written, **assemble everything** (title, intro, sections, conclusion) into a final complete guide file.

4. **Extra utilities**:  
   - `kickoff()` starts the flow when you run the file.
   - `plot()` can generate a visual map (diagram) of how the flow is structured.

#### Key Ideas
- **Flows** organize different actions in a smart, connected way.
- **@start** defines the first step; **@listen** hooks up the next steps automatically when the previous one finishes.
- **State** keeps the memory of what's going on across steps.
- **Direct LLM calls** are used for creative tasks (like outlining).
- **Crews** (small AI teams) are used for heavier tasks (like writing full sections).

## Let's now explain the code line by line

```python
#!/usr/bin/env python
```
- This tells the system that this file should be run with Python.

```python
import json
import os
from typing import List, Dict
from pydantic import BaseModel, Field
from crewai import LLM
from crewai.flow.flow import Flow, listen, start
from guide_creator_flow.crews.content_crew.content_crew import ContentCrew
```
- **Imports**: We bring in necessary libraries:  
  - `json` and `os` for working with files and folders.
  - `typing` for type hints like `List` and `Dict`.
  - `pydantic` for defining structured data models.
  - `crewai` stuff (LLM, Flow, listen, start) for building the AI flow.
  - `ContentCrew` is the crew we just created. We are importing it from `src/guide_creator_flow/crews/content_crew/content_crew.py`

---

#### Define the data models
```python
class Section(BaseModel):
    title: str = Field(description="Title of the section")
    description: str = Field(description="Brief description of what the section should cover")
```
- Defines a **Section**: each part of the guide has a **title** and **description**.

```python
class GuideOutline(BaseModel):
    title: str = Field(description="Title of the guide")
    introduction: str = Field(description="Introduction to the topic")
    target_audience: str = Field(description="Description of the target audience")
    sections: List[Section] = Field(description="List of sections in the guide")
    conclusion: str = Field(description="Conclusion or summary of the guide")
```
- Defines the **full guide structure**: title, intro, audience, multiple sections, and a conclusion.

---

#### Define the flow's memory (state)
```python
class GuideCreatorState(BaseModel):
    topic: str = ""
    audience_level: str = ""
    guide_outline: GuideOutline = None
    sections_content: Dict[str, str] = {}
```
- Defines what the **flow remembers** as it runs:
  - The topic
  - The audience
  - The guide's outline
  - The full content of each section

---

#### Define the flow itself
```python
class GuideCreatorFlow(Flow[GuideCreatorState]):
```
- Creates the **flow** GuideCreatorFlow using the Flow class and `GuideCreatorState`. 

---

## First step: Ask the user
```python
    @start()
    def get_user_input(self):
```
- **@start** marks this as the **first step** of the flow.

```python
        print("\n=== Create Your Comprehensive Guide ===\n")
```
- Prints a welcome message.

```python
        self.state.topic = input("What topic would you like to create a guide for? ")
```
- **Ask** the user for the guide topic and **save** it in the state (state.topic).

```python
        while True:
            audience = input("Who is your target audience? (beginner/intermediate/advanced) ").lower()
            if audience in ["beginner", "intermediate", "advanced"]:
                self.state.audience_level = audience
                break
            print("Please enter 'beginner', 'intermediate', or 'advanced'")
```
- **Ask** the user for the target audience, but **force** them to choose a valid option.


#### Let's explain the previous loop in more detail

```python
while True:
```
- This starts a **loop**.  
- The idea is: **keep asking** the user *over and over* until they give a valid answer.
- It will only stop (`break`) once the user input is correct.

---

```python
audience = input("Who is your target audience? (beginner/intermediate/advanced) ").lower()
```
- **Ask the user** a question:  
  *"Who is your target audience?"*
- `input()` gets what the user types.
- `.lower()` makes the answer **lowercase** no matter how they type it (e.g., "Beginner", "BEGINNER", "beginner" → all become `"beginner"`).
  - This helps avoid mismatch due to case sensitivity.

---

```python
if audience in ["beginner", "intermediate", "advanced"]:
```
- **Check if** the user's answer is **one of the allowed options**:
  - beginner
  - intermediate
  - advanced
- If yes, **accept** it.

---

```python
    self.state.audience_level = audience
    break
```
- **Save** the user's answer into the flow's memory (`self.state.audience_level`).
- **Exit** the `while` loop with `break`.
  - No more asking — move on to the next step.

---

```python
print("Please enter 'beginner', 'intermediate', or 'advanced'")
```
- If the user gave an invalid answer, **this line runs**.
- It **shows a warning message**:  
  *"Hey, you have to pick one of these valid choices."*
- After this, **the loop restarts**, and the user is asked again.


#### Summary of what's happening
1. The loop **keeps asking** until the user gives a valid input.
2. It **forces** correct answers (no typos allowed).
3. It **normalizes** the input to lowercase to make matching easier.
4. Once valid, it **saves** the answer and **continues**.

This loop is **important** because it guarantees that when we later generate a guide, we always know the audience is a valid, expected value.

---

#### Let's continue explaining the rest of the code:


```python
        print(f"\nCreating a guide on {self.state.topic} for {self.state.audience_level} audience...\n")
        return self.state
```
- Print a confirmation message and **return** the current state.

---

## Second step: Generate the guide outline
```python
    @listen(get_user_input)
    def create_guide_outline(self, state):
```
- **@listen(get_user_input)** means:  
  After getting the user input, **do this next**.

```python
        print("Creating guide outline...")
```
- Print a message.

```python
        llm = LLM(model="openai/gpt-4o-mini", response_format=GuideOutline)
```
- Set up the **LLM** (like GPT-4o-mini) to **output the guide with the GuideOutline format**.

```python
        messages = [
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": f""" 
            Create a detailed outline for a comprehensive guide on "{state.topic}" for {state.audience_level} level learners.

            The outline should include:
            1. A compelling title for the guide
            2. An introduction to the topic
            3. 4-6 main sections that cover the most important aspects of the topic
            4. A conclusion or summary

            For each section, provide a clear title and a brief description of what it should cover.
            """}
        ]
```
- Prepare the **prompt** for the LLM, asking it to create an outline with title, intro, 4–6 sections, and a conclusion.

```python
        response = llm.call(messages=messages)
```
- **Call** the LLM with the prompt and get a **response** in JSON format.

```python
        outline_dict = json.loads(response)
```
- **Parse** the JSON string into a **Python dictionary**.

```python
        self.state.guide_outline = GuideOutline(**outline_dict)
```
- **Load** the dictionary into a `GuideOutline` object and **save** it in the state.

**What's happening here, in simple steps:**

1. **`outline_dict`** is a **Python dictionary**.  
   - It comes from parsing the JSON response from the LLM.
   - Example of what `outline_dict` might look like:
     ```python
     {
         "title": "Learn Python from Scratch",
         "introduction": "This guide will introduce Python programming...",
         "target_audience": "beginner",
         "sections": [
             {"title": "Getting Started", "description": "How to set up Python"},
             {"title": "Basic Syntax", "description": "Understanding Python syntax"},
         ],
         "conclusion": "Summary and next steps."
     }
     ```

2. **`GuideOutline(**outline_dict)`** creates a new `GuideOutline` object by **unpacking** the dictionary into the model.
   - The `**` syntax in Python **unpacks** a dictionary:  
     It passes each key-value pair as a separate argument to `GuideOutline`.
   - So, it's the same as doing:
     ```python
     GuideOutline(
         title="Learn Python from Scratch",
         introduction="This guide will introduce Python programming...",
         target_audience="beginner",
         sections=[
             Section(title="Getting Started", description="How to set up Python"),
             Section(title="Basic Syntax", description="Understanding Python syntax"),
         ],
         conclusion="Summary and next steps."
     )
     ```
   - **Notice**:  
     Because we defined `GuideOutline` and `Section` as Pydantic models, **Pydantic automatically handles**:
     - Checking types (e.g., `sections` must be a list of `Section`)
     - Creating nested models correctly (it will create `Section` objects from dictionaries)

3. **`self.state.guide_outline = ...`** saves this new `GuideOutline` object into the flow's memory (state).
   - So now `self.state` holds:
     - The topic
     - The audience level
     - **And now the full structured guide outline**, ready for the next step!


**In simple words:**
- We take the AI's outline (which is in raw dictionary format),
- We "transform" it into a clean, strongly typed `GuideOutline` object,
- We **save** that structured object into the current memory (`state`).


#### Let's continue explaining the rest of the code

```python
        os.makedirs("output", exist_ok=True)
```
- Make an **output** folder if it doesn't exist.

```python
        with open("output/guide_outline.json", "w") as f:
            json.dump(outline_dict, f, indent=2)
```
- Save the outline into a **JSON file**.

```python
        print(f"Guide outline created with {len(self.state.guide_outline.sections)} sections")
        return self.state.guide_outline
```
- Print a success message and **return** the outline.

---

## Third step: Write each section
```python
    @listen(create_guide_outline)
    def write_and_compile_guide(self, outline):
```
- **@listen(create_guide_outline)** means:  
  After creating the outline, **now write the guide**.

```python
        print("Writing guide sections and compiling...")
        completed_sections = []
```
- Print a message and set up an empty list to track completed sections.

```python
        for section in outline.sections:
```
- Loop through **each section** in the outline.

```python
            print(f"Processing section: {section.title}")
```
- Print which section is being written.

```python
            previous_sections_text = ""
            if completed_sections:
                previous_sections_text = "# Previously Written Sections\n\n"
                for title in completed_sections:
                    previous_sections_text += f"## {title}\n\n"
                    previous_sections_text += self.state.sections_content.get(title, "") + "\n\n"
            else:
                previous_sections_text = "No previous sections written yet."
```
- Build context from **previous sections** (if any).

#### Let's explain the previous code block in more detail

First, **what is this code trying to do?**

✅ It builds a **string** that contains the **content already written** for previous sections.  
✅ This way, when writing a new section, the AI knows **what has already been said**.  
✅ It **helps maintain flow** between sections (no repetition, better transitions).

Now, let's review it **line by line**:

```python
previous_sections_text = ""
```
- **Initialize** an empty string.
- This will **hold** the text of previously written sections (if any).

---

```python
if completed_sections:
```
- **Check if** there are any previously completed sections.
- `completed_sections` is a **list** (e.g., `["Introduction", "Getting Started"]`).
- If the list is **not empty**, that means we already wrote something.

---

**If we have previous sections:**

```python
    previous_sections_text = "# Previously Written Sections\n\n"
```
- **Start** the text with a heading called "**Previously Written Sections**".
- Add **two new lines** (`\n\n`) for better Markdown formatting (a blank line).

---

```python
    for title in completed_sections:
```
- **Loop** over each completed section title.

Example: if `completed_sections = ["Introduction", "Getting Started"]`, it loops two times.

---

Inside the loop:

```python
        previous_sections_text += f"## {title}\n\n"
```
- For each section title:
  - Add a subheading (`##`) with the section's title.
  - Again, add two new lines after the heading.

Example:
```
## Introduction

```

---

```python
        previous_sections_text += self.state.sections_content.get(title, "") + "\n\n"
```
- **Add** the actual **content** that was written for that section:
  - `self.state.sections_content` is a dictionary like `{ "Introduction": "Welcome to the guide..." }`.
  - `.get(title, "")` gets the text for that title safely (returns empty string if missing).
- Add **two more new lines** after the content.

Resulting example:
```
## Introduction

Welcome to the guide...

```

---

**If there are NO completed sections yet:**

```python
else:
    previous_sections_text = "No previous sections written yet."
```
- If no sections have been written:
  - The context is simply a sentence saying:  
    `"No previous sections written yet."`

---

### Summary of what this block does
- If **some sections** have already been written:
  - Build a structured context with all their titles and contents.
  - This is formatted nicely in Markdown style (`#`, `##`, newlines).
- If **nothing has been written yet**:
  - Just say "No previous sections written yet."

This **helps the AI** understand:
- What’s been covered already
- How to maintain smooth, logical writing


#### OK. Let's continue explaining the rest of the code.

```python
            result = ContentCrew().crew().kickoff(inputs={
                "section_title": section.title,
                "section_description": section.description,
                "audience_level": self.state.audience_level,
                "previous_sections": previous_sections_text,
                "draft_content": ""
            })
```
- Use the **ContentCrew** to **write the section**, giving it all the needed info.

```python
            self.state.sections_content[section.title] = result.raw
            completed_sections.append(section.title)
            print(f"Section completed: {section.title}")
```
- **Save** the written section, **mark** it as completed, and print a success message.

---

#### Compile everything into one file
```python
        guide_content = f"# {outline.title}\n\n"
        guide_content += f"## Introduction\n\n{outline.introduction}\n\n"
```
- Start building the final guide with title and introduction.

```python
        for section in outline.sections:
            section_content = self.state.sections_content.get(section.title, "")
            guide_content += f"\n\n{section_content}\n\n"
```
- Add all **written sections** in order.

```python
        guide_content += f"## Conclusion\n\n{outline.conclusion}\n\n"
```
- Add the **conclusion**.

```python
        with open("output/complete_guide.md", "w") as f:
            f.write(guide_content)
```
- **Save** the final guide as a **Markdown (.md)** file.

```python
        print("\nComplete guide compiled and saved to output/complete_guide.md")
        return "Guide creation completed successfully"
```
- Print a success message.

---

#### Helper functions
```python
def kickoff():
    """Run the guide creator flow"""
    GuideCreatorFlow().kickoff()
    print("\n=== Flow Complete ===")
    print("Your comprehensive guide is ready in the output directory.")
    print("Open output/complete_guide.md to view it.")
```
- **Starts** the flow and shows a final message.

```python
def plot():
    """Generate a visualization of the flow"""
    flow = GuideCreatorFlow()
    flow.plot("guide_creator_flow")
    print("Flow visualization saved to guide_creator_flow.html")
```
- **Creates a visual map** of how the steps connect.

---

### Finally, when the script runs:
```python
if __name__ == "__main__":
    kickoff()
```
- If you **run the file**, it automatically **starts** the flow.

---

#### In short:  
The code **asks the user for a topic**, **creates a guide plan**, **writes each part**, and **saves** the result — all automatically by combining **user input**, **LLMs**, and **crews**, inside an event-driven **flow**.

## As you saw, configuring a Flow means writing custom Python code
* But do not panick if you are a beginner in Python: once you have seen a few Flows, you will see many common patterns you can reuse or adapt.

## Step 6: Make sure you have the environment variables in the .env file
* `OPENAI_API_KEY=your_openai_api_key`

## Step 7: Install dependencies
In terminal:
* `crewai install`

## Step 8: Run the Flow
In terminal:
* `crewai flow kickoff`

When you run this command, you’ll see your flow at work:
* It will prompt you for a topic and audience level.
* It will create a structured outline for your guide.
* It will process each section, with the content writer and reviewer collaborating on each.
* Finally, it will compile everything into a comprehensive guide.

## Step 9: Visualize the Flow
In terminal:
* `crewai flow plot`

This will create an HTML file that shows the structure of your flow, including the relationships between different steps and the data that flows between them. 

This visualization can be invaluable for understanding and debugging complex flows.

## Step 10: Review the Output
Once the flow completes, you’ll find two files in the output directory:
* `guide_outline.json`: Contains the structured outline of the guide
* `complete_guide.md`: The comprehensive guide with all sections

You can open `complete_guide.md` using JupyterLab. In **JupyterLab** (modern versions 3.x or 4.x), to view a `.md` file in **"Rich Text"** (rendered nicely), you do the following:

1. **Open the `.md` file** normally (double-click it in the file browser on the left).
2. You'll see it open in a **plain text editor**.
3. **Right-click on the tab** of the opened file (at the top).
4. In the context menu that appears, select **"Show Rendered Markdown"**.

👉 Then, instead of seeing the raw Markdown source (`# Title`, `**bold**`, etc.), you’ll directly see the nicely formatted result, like in a web page.

**Important notes:**
- You can switch back to plain text mode by right-clicking again and selecting **"Show Source"**.
- If you don’t see that option, you may need to update JupyterLab or install the official Markdown renderer extension.

## Note about the cost of implementing this project
* To run this Flow costed us $0.01

## Understanding the Flow Structure
Let’s break down the key components of flows to help you understand how to build your own.

#### 1. Flows can make direct calls to the LLM

In [ ]:
llm = LLM(model="openai/gpt-4o-mini", response_format=GuideOutline)
response = llm.call(messages=messages)

#### 2. Flows use decorators to set the logic
This is called "event-driven" architecture.

In [ ]:
@start()
def get_user_input(self):
    # First step in the flow
    # ...

@listen(get_user_input)
def create_guide_outline(self, state):
    # This runs when get_user_input completes
    # ...

#### 3. Flows have memory
This memory is called state, and it persists across steps.

In [ ]:
class GuideCreatorState(BaseModel):
    topic: str = ""
    audience_level: str = ""
    guide_outline: GuideOutline = None
    sections_content: Dict[str, str] = {}

#### 4. Flows integrate Crews easily

In [ ]:
result = ContentCrew().crew().kickoff(inputs={
    "section_title": section.title,
    # ...
})